# NYC Restaurant Inspection - Exploratory Data Analysis

This notebook explores the DOHMH Restaurant Inspection dataset to understand:
1. Data structure and quality
2. Inspection outcome distributions
3. Patterns by cuisine, borough, and time
4. Feature engineering opportunities

**Author:** [Shril Patel]  
**Date:** [Dec 13,2025]

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Settings
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

%matplotlib inline

## 1. Load Data

In [ ]:
# Load the data
# Update this path to your actual file location
DATA_PATH = Path('../data/raw/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

# Load with appropriate dtypes
df = pd.read_csv(
    DATA_PATH,
    low_memory=False,
    dtype={'ZIPCODE': str, 'PHONE': str}
)

# Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
# First look at the data
df.head()

In [ ]:
# Column info
df.info()

## 2. Data Quality Assessment

In [ ]:
# Missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing %', ascending=False)

missing_df[missing_df['Missing Count'] > 0]

In [ ]:
# Visualize missing values
fig, ax = plt.subplots(figsize=(12, 6))
missing_df[missing_df['Missing %'] > 0]['Missing %'].plot(
    kind='barh', ax=ax, color='coral'
)
ax.set_xlabel('Missing %')
ax.set_title('Missing Values by Column')
plt.tight_layout()

## 3. Target Variable Analysis

In [ ]:
# Convert score to numeric
df['score'] = pd.to_numeric(df['score'], errors='coerce')

# Score distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
df['score'].hist(bins=50, ax=axes[0], color='steelblue', edgecolor='white')
axes[0].set_xlabel('Score')
axes[0].set_ylabel('Count')
axes[0].set_title('Inspection Score Distribution')
axes[0].axvline(x=14, color='orange', linestyle='--', label='A/B boundary')
axes[0].axvline(x=28, color='red', linestyle='--', label='B/C boundary')
axes[0].legend()

# Grade distribution
grade_counts = df['grade'].value_counts()
grade_counts.plot(kind='bar', ax=axes[1], color=['green', 'orange', 'red', 'gray', 'gray'])
axes[1].set_xlabel('Grade')
axes[1].set_ylabel('Count')
axes[1].set_title('Grade Distribution')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()

## 4. Borough Analysis

In [ ]:
# Create failure indicator
df['failed'] = (df['score'] >= 14).astype(int)

# Borough stats
borough_stats = df.groupby('boro').agg({
    'camis': 'nunique',
    'score': 'mean',
    'failed': 'mean'
}).round(3)
borough_stats.columns = ['Unique Restaurants', 'Avg Score', 'Fail Rate']
borough_stats.sort_values('Fail Rate', ascending=False)

In [ ]:
# Visualize by borough
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

borough_stats['Unique Restaurants'].sort_values().plot(
    kind='barh', ax=axes[0], color='steelblue'
)
axes[0].set_xlabel('Number of Restaurants')
axes[0].set_title('Restaurants by Borough')

(borough_stats['Fail Rate'] * 100).sort_values().plot(
    kind='barh', ax=axes[1], color='coral'
)
axes[1].set_xlabel('Failure Rate (%)')
axes[1].set_title('Failure Rate by Borough')

plt.tight_layout()

## 5. Cuisine Analysis

In [ ]:
# Top cuisines
top_cuisines = df['cuisine_description'].value_counts().head(15)
print("Top 15 Cuisines:")
print(top_cuisines)

In [ ]:
# Cuisine risk analysis
cuisine_stats = df.groupby('cuisine_description').agg({
    'camis': 'nunique',
    'failed': 'mean'
}).reset_index()
cuisine_stats.columns = ['Cuisine', 'Count', 'Fail Rate']

# Filter for cuisines with at least 100 restaurants
cuisine_stats = cuisine_stats[cuisine_stats['Count'] >= 100].sort_values('Fail Rate', ascending=False)

# Plot top 20 riskiest cuisines
fig, ax = plt.subplots(figsize=(10, 8))
cuisine_stats.head(20).set_index('Cuisine')['Fail Rate'].plot(
    kind='barh', ax=ax, color='coral'
)
ax.set_xlabel('Failure Rate')
ax.set_title('Top 20 Highest-Risk Cuisines')
plt.tight_layout()

## 6. Temporal Analysis

In [ ]:
# Convert dates
df['inspection_date'] = pd.to_datetime(df['inspection_date'], errors='coerce')

# Filter valid dates
df_dated = df[df['inspection_date'].notna()].copy()

# Extract date features
df_dated['year'] = df_dated['inspection_date'].dt.year
df_dated['month'] = df_dated['inspection_date'].dt.month
df_dated['day_of_week'] = df_dated['inspection_date'].dt.day_name()

print(f"Date range: {df_dated['inspection_date'].min()} to {df_dated['inspection_date'].max()}")

In [ ]:
# Monthly trends
monthly = df_dated.groupby(df_dated['inspection_date'].dt.to_period('M')).agg({
    'camis': 'count',
    'failed': 'mean'
})
monthly.columns = ['Count', 'Fail Rate']
monthly.index = monthly.index.astype(str)

# Plot recent 3 years
recent = monthly.tail(36)

fig, ax1 = plt.subplots(figsize=(14, 5))
ax2 = ax1.twinx()

recent['Count'].plot(kind='bar', ax=ax1, color='steelblue', alpha=0.7, width=0.8)
recent['Fail Rate'].plot(ax=ax2, color='coral', linewidth=2, marker='o', markersize=4)

ax1.set_xlabel('Month')
ax1.set_ylabel('Inspection Count', color='steelblue')
ax2.set_ylabel('Failure Rate', color='coral')
ax1.set_title('Monthly Inspection Volume and Failure Rate')

# Show fewer x-tick labels
ax1.set_xticks(range(0, len(recent), 3))
ax1.set_xticklabels(recent.index[::3], rotation=45, ha='right')

plt.tight_layout()

In [ ]:
# Seasonal pattern
seasonal = df_dated.groupby('month')['failed'].mean()

fig, ax = plt.subplots(figsize=(10, 5))
seasonal.plot(kind='bar', ax=ax, color='steelblue')
ax.set_xlabel('Month')
ax.set_ylabel('Failure Rate')
ax.set_title('Seasonal Pattern in Inspection Failures')
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=0)
plt.tight_layout()

## 7. Violation Analysis

In [ ]:
# Critical vs non-critical violations
if 'critical_flag' in df.columns:
    critical_counts = df['critical_flag'].value_counts()
    print("Critical Flag Distribution:")
    print(critical_counts)
    print(f"\nCritical violation rate: {(df['critical_flag'] == 'Critical').mean()*100:.1f}%")

In [ ]:
# Most common violation codes
if 'violation_code' in df.columns:
    top_violations = df['violation_code'].value_counts().head(20)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    top_violations.plot(kind='barh', ax=ax, color='steelblue')
    ax.set_xlabel('Count')
    ax.set_title('Top 20 Most Common Violation Codes')
    plt.tight_layout()

## 8. Key Findings

### Summary of Insights:

1. **Target Variable:**
   - [Add your findings about score/grade distribution]
   - Overall failure rate: XX%

2. **Geographic Patterns:**
   - [Which boroughs have highest/lowest failure rates?]
   - [Any notable zipcode patterns?]

3. **Cuisine Patterns:**
   - [Which cuisines are highest risk?]
   - [Any surprising findings?]

4. **Temporal Patterns:**
   - [Is there seasonality?]
   - [Summer vs winter differences?]

5. **Feature Engineering Ideas:**
   - Historical failure rate per restaurant
   - Days since last inspection
   - Cuisine risk score
   - Borough risk score
   - Seasonal indicators
   - Previous inspection score/outcome

---

## Next Steps

1. Run `src/data_loader.py` to clean and prepare data
2. Run `src/feature_engineering.py` to create features
3. Run `src/model_training.py` to train and evaluate models
4. Launch Streamlit app with `streamlit run streamlit_app/app.py`